In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [24]:
file_1="Dataset/B0005_dis_dataset.csv"
dataset=pd.read_csv(file_1)

In [25]:
dataset.describe()

,cycle,capacity,voltage_measured,current_measured,temperature_measured,current_load,voltage_load,time
count,50285.000000,50285.000000,50285.000000,50285.000000,50285.000000,50285.000000,50285.000000,50285.000000
mean,88.125942,1.560345,3.515268,-1.806032,32.816991,1.362700,2.308406,1546.208924
std,45.699687,0.182380,0.231778,0.610502,3.987515,1.313698,0.800300,906.640295
min,1.000000,1.287453,2.455679,-2.029098,23.214802,-1.998400,0.000000,0.000000
25%,50.000000,1.386229,3.399384,-2.013415,30.019392,1.998000,2.388000,768.563000
50%,88.000000,1.538237,3.511664,-2.012312,32.828944,1.998200,2.533000,1537.031000
75%,127.000000,1.746871,3.660903,-2.011052,35.920887,1.998200,2.690000,2305.984000
max,168.000000,1.856487,4.222920,0.007496,41.450232,1.998400,4.238000,3690.234000


In [44]:
C_max=2

#renaming columns for simplicity
dataset.columns=['cycle',"capacity",'voltage', 'current','temperature', 'current_load', 'voltage_load',"time"]

#removing discharge current rows/avoiding negative current

df=dataset[dataset.voltage>=2.7].reset_index(drop=True)

# converting time seconds->hour

df["time"]=df["time"]/3600

In [45]:
df[df.cycle==1]

,cycle,capacity,voltage,current,temperature,current_load,voltage_load,time
0,1,1.856487,4.191492,-0.004902,24.330034,-0.0006,0.000,0.000000
1,1,1.856487,4.190749,-0.001478,24.325993,-0.0006,4.206,0.004661
2,1,1.856487,3.974871,-2.012528,24.389085,-1.9982,3.062,0.009918
3,1,1.856487,3.951717,-2.013979,24.544752,-1.9982,3.030,0.014939
4,1,1.856487,3.934352,-2.011144,24.731385,-1.9982,3.011,0.019978
...,...,...,...,...,...,...,...,...
191,1,1.856487,3.258705,-0.001513,35.296711,-0.0006,0.000,1.002387
192,1,1.856487,3.264121,-0.001532,35.021738,-0.0006,0.000,1.008042
193,1,1.856487,3.269030,-0.000406,34.754871,-0.0006,0.000,1.013715
194,1,1.856487,3.273207,-0.000388,34.490078,-0.0006,0.000,1.019410


In [50]:
#calculate soc for each cycle
cycle_count=df["cycle"].unique()
soc_list=[]
# Update SOC for each reading within each cycle using Coulomb counting
for cycle in cycle_count:
    cycle_data = df[df['cycle'] == cycle]
    
    cumulative_soc = [0.0]
    
    for i in range(1, len(cycle_data)):
        
        delta_time = cycle_data['time'].iloc[i] - cycle_data['time'].iloc[i-1]
        
        delta_charge = cycle_data['current'].iloc[i] * delta_time
        
        cumulative_soc.append(cumulative_soc[-1] +delta_charge)
        
    soc_list.extend(cumulative_soc)
    
DOD = [1+(q / C_max) for q in soc_list]

df["DOD"]=DOD

In [51]:
df

,cycle,capacity,voltage,current,temperature,current_load,voltage_load,time,DOD
0,1,1.856487,4.191492,-0.004902,24.330034,-0.0006,0.000,0.000000,1.000000
1,1,1.856487,4.190749,-0.001478,24.325993,-0.0006,4.206,0.004661,0.999997
2,1,1.856487,3.974871,-2.012528,24.389085,-1.9982,3.062,0.009918,0.994708
3,1,1.856487,3.951717,-2.013979,24.544752,-1.9982,3.030,0.014939,0.989651
4,1,1.856487,3.934352,-2.011144,24.731385,-1.9982,3.011,0.019978,0.984584
...,...,...,...,...,...,...,...,...,...
50112,168,1.325079,3.579262,-0.001569,34.864823,0.0006,0.000,0.772587,0.338635
50113,168,1.325079,3.581964,-0.003067,34.814770,0.0006,0.000,0.775295,0.338631
50114,168,1.325079,3.584484,-0.003079,34.676258,0.0006,0.000,0.778008,0.338627
50115,168,1.325079,3.587336,0.001219,34.565580,0.0006,0.000,0.780733,0.338629
